In [1]:
#Setup & Imports

import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path

In [2]:
# Load data

df = pd.read_csv("../data/raw/decisions_analysis_ready.csv")

In [6]:
# -----------------------------
# 1. Choice Distribution by Treatment & Evidence Strength (stacked bar)
# -----------------------------

# Compute proportion of each chosen_option per treatment and evidence
choice_dist = (
    df.groupby(["treatment_group", "evidence_strength"])["chosen_option"]
      .value_counts(normalize=True)
      .rename("proportion")
      .reset_index()
)

# Pivot so columns are chosen_option
choice_dist_pivot = choice_dist.pivot_table(
    index=["treatment_group", "evidence_strength"],
    columns="chosen_option",
    values="proportion",
    fill_value=0
)

# Reorder evidence strength
evidence_order = ["High", "Moderate", "Low"]
choice_dist_pivot = choice_dist_pivot.reindex(
    pd.MultiIndex.from_product(
        [choice_dist_pivot.index.levels[0], evidence_order],
        names=choice_dist_pivot.index.names
    )
)

# Plot one stacked bar per treatment
for treatment in choice_dist_pivot.index.levels[0]:
    subset = choice_dist_pivot.loc[treatment]
    subset.plot(kind="bar", stacked=True)
    plt.title(f"Choice Distribution by Evidence Strength")
    plt.ylabel("Proportion")
    plt.xlabel("Evidence Strength")
    plt.legend(["Control", "Treatment"], title="Chosen Option")
    plt.tight_layout()
    plt.savefig("../figures/choice_distribution_treatment.png")
    plt.close()


In [11]:
# 2. Decision Time by Treatment Group (Boxplot)

plt.figure()
df.boxplot(column="decision_time_sec", by="treatment_group")
plt.title("Decision Time by Treatment Group")
plt.suptitle("")
plt.xlabel("Treatment Group")
plt.xticks([1,2], labels = ["Control", "Treatment"])
plt.ylabel("Decision Time (seconds)")
plt.tight_layout()
plt.savefig("../figures/decision_time_by_treatment.png")
plt.close()

<Figure size 640x480 with 0 Axes>

In [16]:
# 3. Optional: Alignment with True Effect (Heatmap-style Table)

alignment_table = (
df.groupby(["treatment_group", "evidence_strength"])["correct_direction"]
.mean()
.unstack()
)


# Reorder evidence strength columns
alignment_table = alignment_table.reindex(columns=["High", "Moderate", "Low"])


plt.figure()
plt.imshow(alignment_table)
plt.colorbar(label="Proportion Correct")
plt.xticks(range(len(alignment_table.columns)), alignment_table.columns)
plt.yticks(range(len(alignment_table.index)), alignment_table.index)
plt.xlabel("Evidence Strength")
plt.ylabel("Treatment Group")
plt.title("Decision Alignment with True Effect (Treatment = 1)")
plt.tight_layout()
plt.savefig("../figures/alignment_heatmap.png")
plt.close()